In [6]:
#Import basic detectron related library
import torch
import torchvision
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

#Import basic library
import numpy as np
import os, json, cv2, random, gc

#Import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import Metadata, DatasetCatalog, MetadataCatalog

In [2]:
!curl -L "https://universe.roboflow.com/ds/yB9A0DttqV?key=LsDpZEWbQ7" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

/bin/bash: /home/hatsu/miniconda3/envs/detectron/lib/libtinfo.so.6: no version information available (required by /bin/bash)
curl: /home/hatsu/miniconda3/envs/detectron/lib/libcurl.so.4: no version information available (required by curl)
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1974      0 --:--:-- --:--:-- --:--:--  1975
100 43.9M  100 43.9M    0     0  9540k      0  0:00:04  0:00:04 --:--:-- 10.4M
Archive:  roboflow.zip
  inflating: README.dataset.txt      
  inflating: README.roboflow.txt     
   creating: test/
 extracting: test/-HD-Best-Moments-in-UFC-Historyframe9_jpg.rf.34eac7014298da0245b7573318e62322.jpg  
 extracting: test/09-Myles-Jury-vs-Michael-Johnson-UFC-155-2418_jpg.rf.9ced7fe80c209ce4816f9bfd08b25297.jpg  
 extracting: test/10-MOST-UNUSUAL-KNOCKOUTSframe14_jpg.rf.7a385883040c74cbbab8020ee6c7034b.jpg  
 extracting: test/

In [2]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

11.7
True


In [3]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, "./train/_annotations.coco.json", "./train")
register_coco_instances("my_dataset_test", {}, "./test/_annotations.coco.json", "./test")

In [4]:
#Visualize the data training
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

for d in random.sample(dataset_dicts, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2.imshow("Visualization", vis.get_image()[:, :, ::-1])
    cv2.waitKey(0)
    cv2.destroyAllWindows()

WARNING [10/22 23:14:17 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



[10/22 23:14:17 d2.data.datasets.coco]: Loaded 700 images in COCO format from ./train/_annotations.coco.json


In [8]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train")
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml")
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.MODEL.DEVICE = "cuda"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=True)
trainer.train()

[10/22 23:18:47 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (4, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (4,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (12, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (12,) in the model! You might want to double check if this is expected.
Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, 

[10/22 23:18:48 d2.engine.train_loop]: Starting training from iteration 0


/home/hatsu/miniconda3/envs/detectron/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /opt/conda/conda-bld/pytorch_1678411187366/work/aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[10/22 23:18:56 d2.utils.events]:  eta: 0:01:51  iter: 19  total_loss: 2.477  loss_cls: 1.406  loss_box_reg: 0.9161  loss_rpn_cls: 0.06086  loss_rpn_loc: 0.03463    time: 0.2316  last_time: 0.2329  data_time: 0.0095  last_data_time: 0.0048   lr: 9.7405e-06  max_mem: 1738M
[10/22 23:19:01 d2.utils.events]:  eta: 0:01:46  iter: 39  total_loss: 2.279  loss_cls: 1.282  loss_box_reg: 0.9439  loss_rpn_cls: 0.05955  loss_rpn_loc: 0.03104    time: 0.2279  last_time: 0.2334  data_time: 0.0058  last_data_time: 0.0054   lr: 1.9731e-05  max_mem: 1738M
[10/22 23:19:05 d2.utils.events]:  eta: 0:01:38  iter: 59  total_loss: 2.024  loss_cls: 1.044  loss_box_reg: 0.9081  loss_rpn_cls: 0.04157  loss_rpn_loc: 0.02584    time: 0.2223  last_time: 0.2246  data_time: 0.0063  last_data_time: 0.0050   lr: 2.972e-05  max_mem: 1738M
[10/22 23:19:09 d2.utils.events]:  eta: 0:01:32  iter: 79  total_loss: 1.737  loss_cls: 0.802  loss_box_reg: 0.8526  loss_rpn_cls: 0.0492  loss_rpn_loc: 0.03611    time: 0.2195  last

In [7]:
del trainer
torch.cuda.empty_cache()
gc.collect()

0